In [1]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer, seed_everything
from src.data import DataModule
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from src.config import radiomics_folder, lesion_level_labels_csv

wandb.init()
wandb.config.aggregation_function = "min"
wandb.config.roi_size = 150
wandb.config.optimizer = "adamw"
wandb.config.weight_decay = 0.00001
wandb.config.model = "SEResNet50"
wandb.config.dropout = 0.07292136035956572
wandb.config.momentum = 0
wandb.config.pretrained = False
wandb.config.learning_rate_max = 0.000023059510738335888
wandb.config.sampler = "stratified"
wandb.config.dim = 2
wandb.config.roi_selection_method = "crop"
# wandb.config.dim = 2
wandb.config.size = 128 if wandb.config.dim == 3 else 256
wandb.config.test_center = None  # "amphia"
wandb.config.lesion_target = "lesion_response"
wandb.config.patient_target = "response"
wandb.config.max_batch_size = 8 if wandb.config.dim == 3 else 64
wandb.config.seed = 0
wandb.config.max_epochs = 100
wandb.config.patience = 10
wandb.config.lr_min = 1e-7
wandb.config.T_0 = 10

seed_everything(wandb.config.seed)

dm = DataModule(
    radiomics_folder,
    lesion_level_labels_csv,
    wandb.config,
)


2023-01-10 15:46:52,407 - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: renstermaat (premium). Use `wandb login --relogin` to force relogin


Global seed set to 0


In [2]:
dm.setup()

train = dm.train_data
val = dm.val_data

In [34]:
import numpy as np

labels = np.array([x['label'] for x in val])
batch_size = 63

last_batch_size = len(labels) % batch_size

if last_batch_size > 0:
    last_batch = list(range(len(labels)))[-last_batch_size:]
    labels_of_ordinary_batches = labels[:-last_batch_size]
else:
    last_batch = []
    labels_of_ordinary_batches = labels

positives = np.where(labels_of_ordinary_batches == 1)[0]
negatives = np.where(labels_of_ordinary_batches == 0)[0]
nans = np.where(np.isnan(labels_of_ordinary_batches))[0]

ordinary_batches = np.concatenate([positives, negatives, nans]).reshape(batch_size, -1).transpose().tolist()



In [35]:
ordinary_batches

[[1,
  20,
  43,
  56,
  74,
  83,
  106,
  160,
  175,
  185,
  201,
  226,
  249,
  277,
  357,
  373,
  401,
  434,
  452,
  464,
  480,
  508,
  518,
  542,
  554,
  563,
  24,
  49,
  84,
  108,
  123,
  133,
  151,
  184,
  214,
  232,
  242,
  263,
  291,
  378,
  403,
  417,
  429,
  461,
  483,
  494,
  532,
  564,
  40,
  93,
  141,
  191,
  253,
  295,
  304,
  313,
  322,
  331,
  340,
  349,
  384,
  393,
  501],
 [2,
  22,
  44,
  57,
  75,
  89,
  107,
  161,
  176,
  186,
  203,
  227,
  250,
  278,
  358,
  374,
  406,
  435,
  453,
  465,
  487,
  509,
  528,
  544,
  555,
  0,
  25,
  51,
  85,
  109,
  124,
  134,
  153,
  204,
  219,
  233,
  247,
  264,
  292,
  379,
  404,
  418,
  440,
  463,
  484,
  495,
  533,
  565,
  41,
  94,
  142,
  192,
  254,
  296,
  305,
  314,
  323,
  332,
  341,
  350,
  385,
  430,
  502],
 [11,
  28,
  45,
  63,
  76,
  95,
  112,
  162,
  177,
  187,
  209,
  229,
  262,
  279,
  359,
  375,
  407,
  436,
  454,
  472,
  488,
 

In [29]:
last_batch

[567, 568, 569, 570, 571, 572, 573, 574, 575]

In [6]:
train_images = [ix['img'] for ix in train]
val_images = [ix['img'] for ix in val]

In [ ]:
[pt for pt in train_patients if pt in val_patients]

In [7]:
[img for img in val_images if img in train_images]

['/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_001_0.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_001_1.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_001_2.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_003_0.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_003_1.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_003_2.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_003_3.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_003_4.nii.gz',
 '/mnt/c/Users/user/data/dl_radiomics/dim-2_size-256_method-crop_roi_size-150/amphia/PREM_AM_005_0.nii.gz',
 '/mnt/c/Users/user/data/dl_

In [12]:
all_batches

[[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0, 1.0, 0.0, 0.0, 0.0, nan, nan],
 [1.0, 1.0,

In [23]:
batch_size = 8
total_batches = np.ceil(len(labels) / batch_size)

len(pos_ix) / total_batches

3.041284403669725

In [29]:
pos_batches = np.array_split(pos_ix, total_batches)
neg_batches = np.array_split(neg_ix, total_batches)

In [32]:
pd.Series([len(x) for x in pos_batches]).value_counts()

3    209
4      9
dtype: int64

In [33]:
pd.Series([len(x) for x in neg_batches]).value_counts()

3    155
2     63
dtype: int64

In [16]:
from pathlib import Path
import nibabel as nib
import os
import numpy as np
from tqdm.notebook import tqdm


r = Path('/mnt/c/users/User/data/dl_radiomics/dim-3_size-182_method-crop_roi_size-71')
for root, _, files in os.walk(r):
    for file in tqdm(files):
        img = nib.load(Path(root) / file)
        new = nib.Nifti1Image(img.get_fdata(), np.eye(4))
        nib.save(new, Path(root) / file)


0it [00:00, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/358 [00:00<?, ?it/s]

  0%|          | 0/328 [00:00<?, ?it/s]

  0%|          | 0/249 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/565 [00:00<?, ?it/s]

  0%|          | 0/495 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

In [17]:
dm.train_transform = None
dm.setup()

batch = next(iter(dm.train_dataloader()))

Loading dataset: 100%|██████████| 581/581 [00:00<00:00, 673732.55it/s]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ToTensord,
    RandFlipd,
    RandRotate90d,
    SaveImaged,
    RandAffined,
    CenterSpatialCropd
)
from src.transforms import RandMirror

transform = Compose([
    LoadImaged(keys=["img"]),
    EnsureChannelFirstd(keys=["img"]),
    # RandMirror(prob=0.5, spatial_axis=0),
    # RandMirror(prob=0.5, spatial_axis=1),
    # RandMirror(prob=0.5, spatial_axis=2),
    RandAffined(keys=['img'], prob=0, rotate_range=[np.pi*2] * 3, padding_mode='zeros'),
    CenterSpatialCropd(keys=['img'], roi_size=(128,128,128))
])

transformed = transform(batch)

In [19]:
import nibabel as nib
from pathlib import Path

imgs = transformed['img']

for img, name in zip(transformed['img'], batch['img']):
    target = Path(name).parent.parent.parent / 'augmented' / Path(name).name
    img = nib.Nifti1Image(img.numpy(), np.eye(4))
    nib.save(img, target)

In [23]:
transformed['img'].shape

(45, 128, 128, 128)

In [16]:
x['img'].permute((0,1,3,2))

tensor([[[[0.2229, 0.2258, 0.2286,  ..., 0.1513, 0.1664, 0.1814],
          [0.2229, 0.2258, 0.2286,  ..., 0.1513, 0.1664, 0.1814],
          [0.2229, 0.2258, 0.2286,  ..., 0.1513, 0.1664, 0.1814],
          ...,
          [0.2237, 0.2235, 0.2234,  ..., 0.0442, 0.0359, 0.0276],
          [0.2237, 0.2235, 0.2234,  ..., 0.0442, 0.0359, 0.0276],
          [0.2237, 0.2235, 0.2234,  ..., 0.0442, 0.0359, 0.0276]],

         [[0.2223, 0.2248, 0.2274,  ..., 0.1428, 0.1563, 0.1699],
          [0.2223, 0.2248, 0.2274,  ..., 0.1428, 0.1563, 0.1699],
          [0.2223, 0.2248, 0.2274,  ..., 0.1428, 0.1563, 0.1699],
          ...,
          [0.2218, 0.2225, 0.2232,  ..., 0.0509, 0.0399, 0.0288],
          [0.2218, 0.2225, 0.2232,  ..., 0.0509, 0.0399, 0.0288],
          [0.2218, 0.2225, 0.2232,  ..., 0.0509, 0.0399, 0.0288]],

         [[0.2217, 0.2239, 0.2261,  ..., 0.1342, 0.1462, 0.1583],
          [0.2217, 0.2239, 0.2261,  ..., 0.1342, 0.1462, 0.1583],
          [0.2217, 0.2239, 0.2261,  ..., 0

In [10]:
from src.transforms import RandTranspose
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ToTensord,
    RandFlipd,
    RandRotate90d,
)

load = LoadImaged(keys=['img'])

x = load(x)

augmentation = Compose(
    [
        # EnsureChannelFirstd(keys=["img"]),
        RandFlipd(keys=["img"], prob=0.5, spatial_axis=0),
        RandFlipd(keys=["img"], prob=0.5, spatial_axis=1),
        RandFlipd(keys=["img"], prob=0.5, spatial_axis=2),
        RandTranspose(),
    ]
)

In [59]:
augmentation = Compose(
        [
            RandFlipd(keys=["img"], prob=0.5, spatial_axis=0),
            RandFlipd(keys=["img"], prob=0.5, spatial_axis=1),
            RandFlipd(keys=["img"], prob=0.5, spatial_axis=2),
            RandTranspose(),
        ]
    )

augmentation(loaded)

{'img': tensor([[[[0.2289, 0.2289, 0.2273,  ..., 0.2655, 0.2637, 0.2637],
           [0.2289, 0.2289, 0.2273,  ..., 0.2655, 0.2637, 0.2637],
           [0.2286, 0.2286, 0.2271,  ..., 0.2667, 0.2649, 0.2649],
           ...,
           [0.2619, 0.2619, 0.2601,  ..., 0.2700, 0.2694, 0.2694],
           [0.2647, 0.2647, 0.2639,  ..., 0.2680, 0.2671, 0.2671],
           [0.2647, 0.2647, 0.2639,  ..., 0.2680, 0.2671, 0.2671]],
 
          [[0.2289, 0.2289, 0.2273,  ..., 0.2655, 0.2637, 0.2637],
           [0.2289, 0.2289, 0.2273,  ..., 0.2655, 0.2637, 0.2637],
           [0.2286, 0.2286, 0.2271,  ..., 0.2667, 0.2649, 0.2649],
           ...,
           [0.2619, 0.2619, 0.2601,  ..., 0.2700, 0.2694, 0.2694],
           [0.2647, 0.2647, 0.2639,  ..., 0.2680, 0.2671, 0.2671],
           [0.2647, 0.2647, 0.2639,  ..., 0.2680, 0.2671, 0.2671]],
 
          [[0.2289, 0.2289, 0.2273,  ..., 0.2655, 0.2637, 0.2637],
           [0.2289, 0.2289, 0.2273,  ..., 0.2655, 0.2637, 0.2637],
           [0.228

In [49]:
from pathlib import Path
import nibabel as nib
import os
from tqdm.notebook import tqdm

r = Path(r'C:\Users\user\data\dl_radiomics')

for d in tqdm(list(r.iterdir())):
    if d.name.startswith('dim-3'):
        for root, _, files in os.walk(d):
            for file in tqdm(files, leave=False):
                img = nib.load(Path(root) / file)
                # print(img.affine)
                new = nib.Nifti1Image(img.get_fdata(), np.eye(4))
                nib.save(new, Path(root) / file)

  0%|          | 0/12 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/174 [00:00<?, ?it/s]

  0%|          | 0/358 [00:00<?, ?it/s]

  0%|          | 0/328 [00:00<?, ?it/s]

...

  0%|          | 0/110 [00:00<?, ?it/s]

  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/565 [00:00<?, ?it/s]

  0%|          | 0/495 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

In [48]:
new.affine

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [12]:
from monai.transforms import LoadImaged, EnsureChannelFirstd, Compose
from src.transforms import (
    FindCentroid,
    GetFixedROISize,
    CropToCentroidWithSize,
    PadForCrop,
    Save,
    OrthogonalSlices,
    CropToROI,
    GetZoomedROI,
    PadForZoom,
)
from pathlib import Path

def find_corresponding_scan(segmention_path):
    scan_folder = Path(segmention_path).parent.parent / "scans"
    scan_name = Path(segmention_path).name.split(".")[0][:-2] + ".nii.gz"

    return scan_folder / scan_name

pipeline = Compose([
    LoadImaged(keys=["img", "seg"]),
    EnsureChannelFirstd(keys=["img", "seg"]),
])


segmentation_path = r'D:\premium_data\lumc\monotherapy\split_segmentations\PREM_LU_009_0.nii.gz'
scan_path = find_corresponding_scan(segmentation_path)
data = {'img':scan_path, 'seg':segmentation_path}

output = pipeline(data)


In [13]:
output['img'].shape

(1, 512, 512, 668)

In [14]:
from src.transforms import GetZoomedROI, PadForZoom, CropToROI

select = Compose(
    [
        GetZoomedROI(0),
        # PadForZoom(),
        # CropToROI()
    ]
)

roi = select(output)

In [15]:
roi['roi_start']

array([210, 257, 522])

In [16]:
roi['roi_end']

array([222, 269, 513])

In [22]:
import numpy as np

w = np.where(roi['seg'])[1:]
bbox_min_coordinates = np.array([axis.min() for axis in w])
bbox_max_coordinates = np.array([axis.max() for axis in w])



In [3]:
from pathlib import Path
import nibabel as nib

r  = Path(r'C:\Users\user\data\dl_radiomics\dim-2_size-256_method-crop_roi_size-50')

for c in r.iterdir():
    for f in c.iterdir():
        shape = nib.load(f).shape
        if not shape == (3,256,256):
            print(f.name, shape)

In [ ]:
dm = Data

In [1]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer, seed_everything
from src.model import Model
from src.data import DataModule
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

wandb.init()

wandb.config.aggregation_function = 'max'
wandb.config.method = 'crop'
wandb.config.dim = 2
wandb.config.size = 256
wandb.config.roi_size= 50
wandb.config.test_center = 'amphia'
wandb.config.lesion_target = 'lung'
wandb.config.max_batch_size = 12
wandb.config.seed = 0
wandb.config.max_epochs = 100
wandb.config.patience = 20
wandb.config.lr = 5e-4,
wandb.config.lr_min = 1e-6,
wandb.config.t_max = 20,

dm = DataModule(
    r"C:\Users\user\data\dl_radiomics",
    r"C:\Users\user\data\tables\lesion_followup_curated_v5.csv",
    wandb.config
)

# model = Model(wandb.config)

# seed_everything(wandb.config.seed)

# logger = WandbLogger(
#     name="hello5",
#     project="project_skeleton_on_lung_lesions",
# )

# checkpoint_callback = ModelCheckpoint(
#     monitor='valid_patient_auc',
#     mode='max'
# )

# early_stopping = EarlyStopping(monitor='valid_patient_auc', mode='max', patience=wandb.config.patience)

# trainer = Trainer(
#     max_epochs=wandb.config.max_epochs, gpus=1, deterministic=True, fast_dev_run=False, logger=logger,
#     checkpoint_callback=checkpoint_callback,
#     callbacks=[
#         early_stopping
#     ]
# )

# trainer.fit(model, dm)

2022-11-15 11:38:49,764 - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: renstermaat. Use `wandb login --relogin` to force relogin


In [3]:
dm.setup()
dl = dm.train_dataloader()

for _ in range(len(dl)):
    try:
        x = next(iter(dl))
    except Exception as e:
        print(e)
        break

Loading dataset: 100%|██████████| 168/168 [00:00<00:00, 167972.13it/s]


In [4]:
for img in dm.train_dataset:
    print(img['img'].shape)

(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 256, 256)
(3, 25

KeyboardInterrupt: 

In [5]:
from pathlib import Path
import os

r = Path(r'D:\premium_data\maxima\monotherapy\split_segmentations')

for file in r.iterdir():
    os.rename(file, file.parent / file.name.replace('.seg.nrrd',''))

In [3]:
wandb.config.t_max

[20]